In [2]:
import os
import json
from google.oauth2 import service_account 
import requests
import zipfile
import pandas as pd
import numpy as np

In [ ]:
import csv
import datetime
import difflib
import glob
import os

import pandas as pd

from datetime import timedelta, datetime

from fuzzywuzzy import fuzz
from fuzzywuzzy import process


In [3]:
bq_project_id = os.environ.get('BQ_PROJECT_ID')
# Establish BigQuery credentials
bq_account_creds = json.loads(os.environ.get('BQ_ACCOUNT_CREDS'))
bq_credentials = service_account.Credentials.from_service_account_info(bq_account_creds, scopes=["https://www.googleapis.com/auth/cloud-platform"])

In [4]:
panda_dtypes_dict = {
    'STRING': 'str',
    'INT64': 'int64',
    'FLOAT64': 'float64',
    'DATETIME': 'datetime'
}

contribution_schema_lst = [
    {'name': 'RecordID', 'type': 'INT64', 'mode': 'REQUIRED'},
    {'name': 'AmendedRecordID', 'type': 'INT64', 'mode': 'NULLABLE'},
    {'name': 'Amended', 'type': 'STRING', 'mode': 'NULLABLE'},
    {'name': 'Amendment', 'type': 'STRING', 'mode': 'NULLABLE'},
    {'name': 'CO_ID', 'type': 'INT64', 'mode': 'REQUIRED'},
    {'name': 'CommitteeType', 'type': 'STRING', 'mode': 'NULLABLE'},
    {'name': 'CommitteeName', 'type': 'STRING', 'mode': 'NULLABLE'},
    {'name': 'CandidateName', 'type': 'STRING', 'mode': 'NULLABLE'},
    {'name': 'ContributionType', 'type': 'STRING', 'mode': 'NULLABLE'},
    {'name': 'ReceiptType', 'type': 'STRING', 'mode': 'NULLABLE'},
    {'name': 'ContributionAmount', 'type': 'FLOAT64', 'mode': 'NULLABLE'},
    {'name': 'ContributionDate', 'type': 'DATETIME', 'mode': 'REQUIRED'},
    {'name': 'FiledDate', 'type': 'DATETIME', 'mode': 'REQUIRED'},
    {'name': 'LastName', 'type': 'STRING', 'mode': 'NULLABLE'},
    {'name': 'FirstName', 'type': 'STRING', 'mode': 'NULLABLE'},
    {'name': 'MI', 'type': 'STRING', 'mode': 'NULLABLE'},
    {'name': 'Suffix', 'type': 'STRING', 'mode': 'NULLABLE'},
    {'name': 'Address1', 'type': 'STRING', 'mode': 'NULLABLE'},
    {'name': 'Address2', 'type': 'STRING', 'mode': 'NULLABLE'},
    {'name': 'City', 'type': 'STRING', 'mode': 'NULLABLE'},
    {'name': 'State', 'type': 'STRING', 'mode': 'NULLABLE'},
    {'name': 'Zip', 'type': 'STRING', 'mode': 'NULLABLE'},
    {'name': 'ContributorType', 'type': 'STRING', 'mode': 'NULLABLE'},
    {'name': 'Employer', 'type': 'STRING', 'mode': 'NULLABLE'},
    {'name': 'Occupation', 'type': 'STRING', 'mode': 'NULLABLE'},
    {'name': 'OccupationComments', 'type': 'STRING', 'mode': 'NULLABLE'},
    {'name': 'Electioneering', 'type': 'STRING', 'mode': 'NULLABLE'},
    {'name': 'Jurisdiction', 'type': 'STRING', 'mode': 'NULLABLE'},
    {'name': 'Explanation', 'type': 'STRING', 'mode': 'NULLABLE'}
]

expenditure_schema_lst = [
    {'name': 'RecordID', 'type': 'INT64', 'mode': 'REQUIRED'},
    {'name': 'AmendedRecordID', 'type': 'INT64', 'mode': 'NULLABLE'},
    {'name': 'Amended', 'type': 'STRING', 'mode': 'NULLABLE'},
    {'name': 'Amendment', 'type': 'STRING', 'mode': 'NULLABLE'},
    {'name': 'CO_ID', 'type': 'INT64', 'mode': 'REQUIRED'},
    {'name': 'CommitteeType', 'type': 'STRING', 'mode': 'NULLABLE'},
    {'name': 'CommitteeName', 'type': 'STRING', 'mode': 'NULLABLE'},
    {'name': 'CandidateName', 'type': 'STRING', 'mode': 'NULLABLE'},
    {'name': 'ExpenditureType', 'type': 'STRING', 'mode': 'NULLABLE'},
    {'name': 'PaymentType', 'type': 'STRING', 'mode': 'NULLABLE'},
    {'name': 'ExpenditureAmount', 'type': 'FLOAT64', 'mode': 'NULLABLE'},
    {'name': 'ExpenditureDate', 'type': 'DATETIME', 'mode': 'REQUIRED'},
    {'name': 'FiledDate', 'type': 'DATETIME', 'mode': 'REQUIRED'},
    {'name': 'LastName', 'type': 'STRING', 'mode': 'NULLABLE'},
    {'name': 'FirstName', 'type': 'STRING', 'mode': 'NULLABLE'},
    {'name': 'MI', 'type': 'STRING', 'mode': 'NULLABLE'},
    {'name': 'Suffix', 'type': 'STRING', 'mode': 'NULLABLE'},
    {'name': 'Address1', 'type': 'STRING', 'mode': 'NULLABLE'},
    {'name': 'Address2', 'type': 'STRING', 'mode': 'NULLABLE'},
    {'name': 'City', 'type': 'STRING', 'mode': 'NULLABLE'},
    {'name': 'State', 'type': 'STRING', 'mode': 'NULLABLE'},
    {'name': 'Zip', 'type': 'STRING', 'mode': 'NULLABLE'},
    {'name': 'DisbursementType', 'type': 'STRING', 'mode': 'NULLABLE'},
    {'name': 'Employer', 'type': 'STRING', 'mode': 'NULLABLE'},
    {'name': 'Occupation', 'type': 'STRING', 'mode': 'NULLABLE'},
    {'name': 'Electioneering', 'type': 'STRING', 'mode': 'NULLABLE'},
    {'name': 'Jurisdiction', 'type': 'STRING', 'mode': 'NULLABLE'},
    {'name': 'Explanation', 'type': 'STRING', 'mode': 'NULLABLE'}
]

loan_schema_lst = [
    {'name': 'RecordID', 'type': 'INT64', 'mode': 'REQUIRED'},
    {'name': 'AmendedRecordID', 'type': 'INT64', 'mode': 'NULLABLE'},
    {'name': 'Amended', 'type': 'STRING', 'mode': 'NULLABLE'},
    {'name': 'Amendment', 'type': 'STRING', 'mode': 'NULLABLE'},
    {'name': 'CO_ID', 'type': 'INT64', 'mode': 'REQUIRED'},
    {'name': 'CommitteeType', 'type': 'STRING', 'mode': 'NULLABLE'},
    {'name': 'CommitteeName', 'type': 'STRING', 'mode': 'NULLABLE'},
    {'name': 'CandidateName', 'type': 'STRING', 'mode': 'NULLABLE'},
    {'name': 'PaymentAmount', 'type': 'FLOAT64', 'mode': 'NULLABLE'},
    {'name': 'PaymentDate', 'type': 'DATETIME', 'mode': 'REQUIRED'},
    {'name': 'FiledDate', 'type': 'DATETIME', 'mode': 'REQUIRED'},
    {'name': 'Name', 'type': 'STRING', 'mode': 'NULLABLE'},
    {'name': 'Address1', 'type': 'STRING', 'mode': 'NULLABLE'},
    {'name': 'Address2', 'type': 'STRING', 'mode': 'NULLABLE'},
    {'name': 'City', 'type': 'STRING', 'mode': 'NULLABLE'},
    {'name': 'State', 'type': 'STRING', 'mode': 'NULLABLE'},
    {'name': 'Zip', 'type': 'STRING', 'mode': 'NULLABLE'},
    {'name': 'Type', 'type': 'STRING', 'mode': 'NULLABLE'},
    {'name': 'Jurisdiction', 'type': 'STRING', 'mode': 'NULLABLE'},
    {'name': 'Description', 'type': 'STRING', 'mode': 'NULLABLE'},
    {'name': 'LoanSourceType', 'type': 'STRING', 'mode': 'NULLABLE'},
    {'name': 'LoanAmount', 'type': 'FLOAT64', 'mode': 'NULLABLE'},
    {'name': 'LoanBalance', 'type': 'FLOAT64', 'mode': 'NULLABLE'},
    {'name': 'LoanDate', 'type': 'DATETIME', 'mode': 'REQUIRED'},
    {'name': 'LoanSource', 'type': 'STRING', 'mode': 'NULLABLE'},
    {'name': 'InterestRate', 'type': 'FLOAT64', 'mode': 'NULLABLE'},
    {'name': 'InterestPayment', 'type': 'FLOAT64', 'mode': 'NULLABLE'},
]

In [5]:
def get_sos_files(category_str: str, end_year_int: int):
    for i in range(1, (end_year_int - 2000)):
        print(f"Collecting {str(2000 + i)} {category_str} data")
        url_stem = 'https://tracer.sos.colorado.gov/PublicSite/Docs/BulkDataDownloads/'
        url = url_stem + str(2000 + i) + '_' + category_str + 'Data.csv.zip'

        req = requests.get(url)
        filename = url.split('/')[-1]

        with open(filename, 'wb') as output_file:
            output_file.write(req.content)
        print(f"Finished downloading {str(2000 + i)} data")
    
    return(print(f"Finished downloading all {category_str} data for 2000 to {str(end_year_int)}"))


In [6]:
def unzip_sos_files(category_str: str, end_year_int: int):
    for i in range(1, (end_year_int - 2000)):
        filename = str(2000 + i) +  '_' + category_str + 'Data.csv.zip'
        print(f"Unzipping {filename}")

        zip_ref = zipfile.ZipFile(filename)
        zip_ref.extractall('.')
        zip_ref.close()
    
    return(f"Finished unzipping all {category_str} data.")

In [7]:
def set_dtypes(df, schema_lst):
    col_dtypes_dict = {x.get('name'): panda_dtypes_dict.get(x.get('type')) for x in schema_lst}
    
    for col in list(df):
        print(f"Setting values on {col} column.")
        try:
            df[col] = df[col].astype('str')
            df = df.sort_values(col)

            if col_dtypes_dict.get(col) == 'datetime':
                date_pre_int = len(df)
                df = df.drop(df.index[df[col] >= '2100-01-01 00:00:00'].tolist())
                print(f"Purged {date_pre_int - len(df)} from column {col}")
                df[col] = pd.to_datetime(df[col])
            elif col_dtypes_dict.get(col) == 'int64':
                df[col] = df[col].replace('nan', '0')
                df[col] = df[col].str.replace('.', '').astype('int64')
            else:
                df[col] = df[col].astype(col_dtypes_dict.get(col), errors='raise')
        except Exception as e:
            if 'nan' in str(e).lower():
                print("Bad nan value")
                df = df.drop(df.index[df[col].isnull()].tolist())

                if col_dtypes_dict.get(col) == 'datetime':
                    df[col] = pd.to_datetime(df[col])
                else:
                    df[col] = df[col].astype(col_dtypes_dict.get(col), errors='ignore')

            elif ('invalid' in str(e)) | ('format' in str(e)) | ('timestamp' in str(e)):
                bad_value = str(e).split(': ')[1]
                print(f"Bad data point, {bad_value} in column {col}. Skipping.")

                if df[col].astype('str').min() == bad_value.replace("'", ""):
                    df = df.drop(df.index[df[col] == bad_value.replace("'", "")])
                    df = df.drop(df.index[df[col] == df[col].astype('str').max()])
                else:
                    df = df.drop(df.index[df[col] >= bad_value.replace("'", "")].tolist())

                if col_dtypes_dict.get(col) == 'datetime':
                    df[col] = pd.to_datetime(df[col])
                else:
                    df[col] = df[col].astype(col_dtypes_dict.get(col), errors='raise')
            else:
                raise
    
    return(df)


In [13]:
def sos_to_gbq(category_str: str, schema_lst: list, end_year_int: int):
    _totals = pd.DataFrame(columns=['year', 'csv', 'uploaded'])

    for i in range(1, (end_year_int - 2000)):
        if i == 1:
            mode_str = 'replace'
        else:
            mode_str = 'append'

        filename = str(2000 + i) + '_' + category_str + 'Data.csv'
        print(f"Reading {filename} to DataFrame.")
        _df = pd.read_csv(filename, encoding='cp437', on_bad_lines='warn', low_memory=False)
        _totals.loc[i, 'year'] = str(2000 + i)
        _totals.loc[i, 'csv'] = len(_df)

        print(f"Setting {filename} data types.")
        _df = set_dtypes(_df, schema_lst)
        display(_df)

        print(f"Uploading {filename} to BigQuery.")
        _df.to_gbq(destination_table='co_campaign_finance.' + category_str.lower(), project_id=bq_project_id, if_exists=mode_str, table_schema=schema_lst, credentials=bq_credentials)
        _totals.loc[i, 'uploaded'] = len(_df)

    return(_totals)

In [ ]:
get_sos_files('Contribution', 2021)

In [ ]:
unzip_sos_files('Contribution', 2021)

In [ ]:
totals_df = sos_to_gbq('Contribution', contribution_schema_lst, 2021)

In [ ]:
totals_df

In [ ]:
get_sos_files('Expenditure', 2021)

In [ ]:
unzip_sos_files('Expenditure', 2021)

In [17]:
totals_df = sos_to_gbq('Expenditure', expenditure_schema_lst, 2021)

Reading 2001_ExpenditureData.csv to DataFrame.
Setting 2001_ExpenditureData.csv data types.
Setting values on CO_ID column.
Setting values on ExpenditureAmount column.
Setting values on ExpenditureDate column.
Purged 1 from column ExpenditureDate
Setting values on LastName column.
Setting values on FirstName column.
Setting values on MI column.


C:\Users\thewh\AppData\Local\Temp\ipykernel_7488\1162268645.py:17: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[col] = df[col].str.replace('.', '').astype('int64')


Setting values on Suffix column.
Setting values on Address1 column.
Setting values on Address2 column.
Setting values on City column.
Setting values on State column.
Setting values on Zip column.
Setting values on Explanation column.
Setting values on RecordID column.
Setting values on FiledDate column.
Purged 0 from column FiledDate
Setting values on ExpenditureType column.


C:\Users\thewh\AppData\Local\Temp\ipykernel_7488\1162268645.py:17: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[col] = df[col].str.replace('.', '').astype('int64')


Setting values on PaymentType column.
Setting values on DisbursementType column.
Setting values on Electioneering column.
Setting values on CommitteeType column.
Setting values on CommitteeName column.
Setting values on CandidateName column.
Setting values on Employer column.
Setting values on Occupation column.
Setting values on Amended column.
Setting values on Amendment column.
Setting values on AmendedRecordID column.
Setting values on Jurisdiction column.


...

C:\Users\thewh\AppData\Local\Temp\ipykernel_7488\1162268645.py:17: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[col] = df[col].str.replace('.', '').astype('int64')


Setting values on Jurisdiction column.


,CO_ID,ExpenditureAmount,ExpenditureDate,LastName,FirstName,MI,Suffix,Address1,Address2,City,...,Electioneering,CommitteeType,CommitteeName,CandidateName,Employer,Occupation,Amended,Amendment,AmendedRecordID,Jurisdiction
6231,20195036475,10.00,2020-02-24,WELLS FARGO,nan,nan,nan,15 S MAIN ST,nan,BRIGHTON,...,nan,Candidate Committee,COVARRUBIAS FOR ADAMS COUNTY,PHILIP COVARRUBIAS,nan,nan,N,N,0,ADAMS
40235,20195036459,252.37,2020-09-14,BANNERS ON THE CHEAP,nan,nan,nan,11525A STONEHOLLOW DR,SUITE 100,AUSTIN,...,YES,Candidate Committee,LYNN FOR ADAMS COUNTY,LYNN BACA,nan,nan,N,N,0,ADAMS
8146,20195036459,35.04,2020-03-08,ACTBLUE,nan,nan,nan,PO BOX 441146,nan,SOMERVILLE,...,nan,Candidate Committee,LYNN FOR ADAMS COUNTY,LYNN BACA,nan,nan,N,N,0,ADAMS
42457,20195036459,598.00,2020-09-21,COLORADO COMMUNITY MEDIA,nan,nan,nan,750 W HAMPDEN AVE,SUITE 225,ENGLEWOOD,...,YES,Candidate Committee,LYNN FOR ADAMS COUNTY,LYNN BACA,nan,nan,N,N,0,ADAMS
55798,20195036459,495.00,2020-10-22,METROWEST NEWSPAPERS,nan,nan,nan,143 SOUTH 2ND PLACE,nan,BRIGHTON,...,YES,Candidate Committee,LYNN FOR ADAMS COUNTY,LYNN BACA,nan,nan,N,N,0,ADAMS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44125,20085632310,224.00,2020-09-26,YUMA PIONEER,nan,nan,nan,207 S MAIN,nan,YUMA,...,nan,Candidate,DEAN WINGFIELD,nan,nan,nan,N,N,0,YUMA
18150,20205038117,68.25,2020-05-29,BLECHA,BETSY,nan,nan,715 JACKSON STREET,nan,WRAY,...,YES,Candidate,BETSY BLECHA,nan,nan,nan,N,N,0,YUMA
33535,20205038117,56.00,2020-08-20,YUMA PIONEER,nan,nan,nan,207 S. MAIN ST,nan,YUMA,...,YES,Candidate,BETSY BLECHA,nan,nan,nan,N,N,0,YUMA
35272,20205037979,136.50,2020-08-27,WHOMBLE,MELINDA,nan,nan,33837 COUNTY ROAD 31,nan,WRAY,...,nan,Candidate,MELINDA JO WHOMBLE,nan,nan,nan,N,N,0,YUMA


Uploading 2020_ExpenditureData.csv to BigQuery.


100%|██████████| 1/1 [00:00<00:00, 999.83it/s]


In [18]:
totals_df

,year,csv,uploaded
1,2001,5939,5938
2,2002,23641,23638
3,2003,6180,6180
4,2004,23740,23738
5,2005,8698,8697
6,2006,45724,45720
7,2007,12284,12284
8,2008,44952,44952
9,2009,14560,14560
10,2010,61909,61871


In [ ]:
get_sos_files('Loan', 2021)

In [ ]:
unzip_sos_files('Loan', 2021)

In [ ]:
totals_df = sos_to_gbq('Loan', loan_schema_lst, 2021)

In [ ]:
totals_df

In [ ]:
loans_df = pd.read_csv('2001_LoanData.csv')

In [ ]:
loans_df[loans_df['PaymentDate'] >= '2100-01-01 00:00:00']


In [ ]:
loans_df.index[loans_df['PaymentDate'] >= '2100-01-01 00:00:00'].tolist()